# Contexto

Este notebook presenta los procesos para obtener una sabana unificada tanto para los datasets de entrenamiento y prueba del challenge de Kaggle "Home Credit - Credit Risk Model Stability". Disponible en:

https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability



## Librerías

In [1]:
import FUNCIONES_AUXILIARES as funciones
import CLASES_AUXILIARES as clases
from pathlib import Path
import numpy as np

## Configuración Incial

In [2]:
ROOT = Path('Data')
TRAIN_DIR = ROOT / 'parquet_files/train'
TEST_DIR = ROOT / 'parquet_files/test'

## Lectura de datos

In [3]:
# Leer información de datos datos de los archivos suministrados

train_data_dict = {'df_base': funciones.read_file(TRAIN_DIR / 'train_base.parquet')
                   ,'depth_0': [funciones.read_file(TRAIN_DIR / 'train_static_cb_0.parquet')
                                ,funciones.read_files(TRAIN_DIR / 'train_static_0_*.parquet')
                                ,
                                ]
                   ,'depth_1': [funciones.read_files(TRAIN_DIR / 'train_applprev_1_*.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_tax_registry_a_1.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_tax_registry_b_1.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_tax_registry_c_1.parquet', 1)
                                ,funciones.read_files(TRAIN_DIR / 'train_credit_bureau_a_1_*.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_credit_bureau_b_1.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_other_1.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_person_1.parquet', 1)
                                ,funciones.read_file(TRAIN_DIR / 'train_deposit_1.parquet', 1)
                                , funciones.read_file(TRAIN_DIR / 'train_debitcard_1.parquet', 1)
                                ,
                                ]
                   ,'depth_2': [funciones.read_file(TRAIN_DIR / 'train_credit_bureau_b_2.parquet', 2)
                                ,funciones.read_files(TRAIN_DIR / 'train_credit_bureau_a_2_*.parquet', 2)
                                ,
                                ]
                   }

# Lo mismo para el conjunto de test

test_data_dict = {'df_base': funciones.read_file(TEST_DIR / 'test_base.parquet')
                  ,'depth_0': [funciones.read_file(TEST_DIR / 'test_static_cb_0.parquet')
                               ,funciones.read_files(TEST_DIR / 'test_static_0_*.parquet')
                               ,
                               ]
                  ,'depth_1': [funciones.read_files(TEST_DIR / 'test_applprev_1_*.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_tax_registry_a_1.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_tax_registry_b_1.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_tax_registry_c_1.parquet', 1)
                               ,funciones.read_files(TEST_DIR / 'test_credit_bureau_a_1_*.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_credit_bureau_b_1.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_other_1.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_person_1.parquet', 1)
                               ,funciones.read_file(TEST_DIR / 'test_deposit_1.parquet', 1)
                               , funciones.read_file(TEST_DIR / 'test_debitcard_1.parquet', 1)
                               ,
                               ]
                  ,'depth_2': [funciones.read_file(TEST_DIR / 'test_credit_bureau_b_2.parquet', 2)
                               ,funciones.read_files(TEST_DIR / 'test_credit_bureau_a_2_*.parquet', 2)
                               ,
                               ]
                  }

## Procesamiento

In [4]:
df_train = funciones.feature_eng(**train_data_dict)
df_test = funciones.feature_eng(**test_data_dict)

print('train data shape:\t'
      , df_train.shape
      , '\n'
      ,'test data shape:\t'
      , df_test.shape
     )

train data shape:	 (1526659, 472) 
 test data shape:	 (10, 471)


In [5]:
# Se aplica el filtro de variables en el conjunto de train y se dejan las mismas columnas para test

df_train = df_train.pipe(clases.Pipeline.filter_cols)
df_test = df_test.select([col for col in df_train.columns if col != "target"])

print('train data shape:\t'
      , df_train.shape
      , '\n'
      ,'test data shape:\t'
      , df_test.shape
     )

train data shape:	 (1526659, 361) 
 test data shape:	 (10, 360)


In [6]:
# Se transforma de polars a pandas

df_train, cat_cols = funciones.to_pandas(df_train)
df_test, cat_cols = funciones.to_pandas(df_test, cat_cols)

In [7]:
# Corregir ariables con formatos errados

columnas_modificar = ['equalitydataagreement_891L'
                      , 'isdebitcard_729L'
                      , 'opencred_647L'
                      , 'max_isbidproduct_390L'
                      , 'max_isdebitcard_527L'
                      , 'max_contaddr_matchlist_1032L'
                      , 'max_contaddr_smempladdr_334L'
                      , 'max_remitter_829L'
                      , 'max_safeguarantyflag_411L'
                     ]

for i in columnas_modificar:
    for df in [df_train, df_test]:
        df[i] = df[i].astype('float')
        df[i] = df[i].apply(lambda x: np.nan if (x != x)
                            else 1 if x == 1
                            else 0
                            )

In [8]:
# Eliminación de diccionarios para liberar memoria

del [train_data_dict, test_data_dict]

In [9]:
## Dataset de Salida

df_train.to_parquet('Data/pre_processed/df_train.parquet')
df_test.to_parquet('Data/pre_processed/df_test.parquet')